In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
!pip install peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


## **Importing packages**

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PromptEncoderConfig,
)
import datasets
from datasets import load_dataset
#import evaluate
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from tqdm import tqdm
from tqdm import trange
from datasets import Dataset
import os
from transformers import EarlyStoppingCallback

model_name_or_path = "xlm-roberta-large"
num_epochs = 20
lr = 1e-3


metric = datasets.load_metric('f1')

<ipython-input-6-d3b4fba35787>:41: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('f1')


## **Reading data files**

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BLP2023/blp_task2/data/blp23_sentiment_train.tsv', sep = '\t')
dev_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BLP2023/blp_task2/data/blp23_sentiment_dev.tsv', sep='\t')
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BLP2023/blp_task2/data/blp23_sentiment_dev_test.tsv', sep='\t')

In [ ]:
if any(k in model_name_or_path for k in ("gpt", "opt", "bloom")):
    padding_side = "left"
else:
    padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, padding_side=padding_side)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
train_data.head()

,id,text,label
0,10856,এখানে আরো ভালো ভাবে দলীয় ও র এর অবস্থান পাকা হ...,Neutral
1,sentinob_1072,চুয়াডাঙ্গা বাড়ি কে বলেছে আপনার,Neutral
2,sentinob_10530,"ভাই সোনাই ঘোষ এর দই খেয়ে যাইতেন , খুব ই মজার",Positive
3,8001,সমার তালুকদার আপনার ছবিতে ফেসটা কেন জানি বন্য ...,Negative
4,sentinob_10144,ভাইয়া এই নুডলস টা কোথায় কিনতে পাওয়া যাবে প্লিজ...,Positive


In [ ]:
label_to_id = {'Positive' : 1, 'Neutral': 0, 'Negative': 2}
id_to_label = {k: v for v, k in label_to_id.items()}
print(id_to_label)

{1: 'Positive', 0: 'Neutral', 2: 'Negative'}


In [ ]:
train_texts = train_data['text'].tolist()
dev_texts = dev_data['text'].tolist()

train_label = train_data['label'].tolist()
train_label_id = [label_to_id[i] for i in train_label]
dev_label = dev_data['label'].tolist()
dev_label_id = [label_to_id[i] for i in dev_label]

In [ ]:
train_data['label'] = train_label_id
dev_data['label'] = dev_label_id
train_data.head()

,id,text,label
0,10856,এখানে আরো ভালো ভাবে দলীয় ও র এর অবস্থান পাকা হ...,0
1,sentinob_1072,চুয়াডাঙ্গা বাড়ি কে বলেছে আপনার,0
2,sentinob_10530,"ভাই সোনাই ঘোষ এর দই খেয়ে যাইতেন , খুব ই মজার",1
3,8001,সমার তালুকদার আপনার ছবিতে ফেসটা কেন জানি বন্য ...,2
4,sentinob_10144,ভাইয়া এই নুডলস টা কোথায় কিনতে পাওয়া যাবে প্লিজ...,1


## **Distribution of positive sentences**

In [ ]:
train_data_pos = train_data[train_data['label'] == 1]
pos_sen = train_data_pos['text'].tolist()
print(len(pos_sen))
p_l = []
for i in range(len(pos_sen)):
    tok = tokenizer.tokenize(pos_sen[i])
    p_l.append(len(tok))

print(max(p_l), min(p_l), sum(p_l)/len(p_l))

12364
3318 1 27.250727919767066


## **Distribution of negative sentences**

In [ ]:
train_data_neg = train_data[train_data['label'] == 2]
neg_sen = train_data_neg['text'].tolist()
print(len(neg_sen))
n_l = []
for i in range(len(neg_sen)):
    tok = tokenizer.tokenize(neg_sen[i])
    n_l.append(len(tok))

print(max(n_l), min(n_l), sum(n_l)/len(n_l))

15767
5366 1 30.425382127227753


## **Distribution of neutral sentences**

In [ ]:
train_data_neu = train_data[train_data['label'] == 0]
neu_sen = train_data_neu['text'].tolist()
print(len(neu_sen))
nu_l = []
for i in range(len(neu_sen)):
    tok = tokenizer.tokenize(neu_sen[i])
    nu_l.append(len(tok))

print(max(nu_l), min(nu_l), sum(nu_l)/len(nu_l))

7135
1002 1 28.296846531184304


In [ ]:
max_len = 200
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions = predictions, references = labels, average ='micro')


In [ ]:
def tokenize_function(examples):
    # max_length=None => use the model max length (it's actually the default)
    outputs = tokenizer(examples["text"], padding = True, truncation= True, max_length= max_len)
    return outputs

In [ ]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(dev_data)

In [ ]:
train_dataset[0]

{'id': '10856',
 'text': 'এখানে আরো ভালো ভাবে দলীয় ও র এর অবস্থান পাকা হলো কি ? ? ',
 'label': 0}

In [ ]:
train_tokenized_datasets = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns = ['id', 'text']
)

train_tokenized_datasets = train_tokenized_datasets.rename_column("label", "labels")


test_tokenized_datasets = test_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns = ['id', 'text']
)

test_tokenized_datasets = test_tokenized_datasets.rename_column("label", "labels")


Map:   0%|          | 0/35266 [00:00<?, ? examples/s]

Map:   0%|          | 0/3934 [00:00<?, ? examples/s]

In [ ]:
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = 'max_length')

In [ ]:
peft_config = PromptEncoderConfig(task_type="SEQ_CLS", num_virtual_tokens=20, encoder_hidden_size=128)

## **Initializing model**

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, num_labels = len(label_to_id), return_dict=True)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)

#print(model.get_prompt(batch_size = 32))
model.print_trainable_parameters()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,405,638 || all params: 561,246,470 || trainable%: 0.4286241657787175


/usr/local/lib/python3.10/dist-packages/peft/tuners/p_tuning.py:146: UserWarning: for MLP, the `encoder_num_layers` is ignored. Exactly 2 MLP layers are used.
  warnings.warn(


## **Defining training arguments**

In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/BLP2023/Saved_dir/P-tuning/XLM_roberta-large",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
early_stop = EarlyStoppingCallback(2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    tokenizer=tokenizer,
    #data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stop]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,1.050800,1.027305,0.468226
2,1.023900,1.127343,0.407728
3,1.016700,0.976460,0.549059
4,1.061900,1.027768,0.495933
5,0.992100,0.967604,0.561515
6,0.966500,0.928910,0.590747
7,0.965000,0.912207,0.595069
8,0.948600,0.893553,0.604474
9,0.936400,0.891053,0.601678
10,0.931400,0.897457,0.596085


FailedPreconditionError: ignored

In [ ]:
trainer.save_model()

## **Loading model for inference**

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "/content/drive/MyDrive/Colab Notebooks/BLP2023/Saved_dir/P-tuning/XLM_roberta-large"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, num_labels = len(label_to_id))
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(inference_model, peft_model_id)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm
test_sentence = test_data['text'].tolist()
label = []
for i in tqdm(range(len(test_sentence))):
    inputs = tokenizer(test_sentence[i], truncation=True, padding='max_length', return_tensors="pt", max_length=100)
    #print(inputs)
    if len(inputs['input_ids']) > 512:
      inputs['input_ids'] = inputs['input_ids'][:512]
      inputs['attention_mask'] = inputs['attention_mask'][:512]

    with torch.no_grad():
        outputs = model(**inputs).logits
        #print(outputs)

    paraphrased_text = torch.softmax(outputs, dim=1).tolist()[0]
    max_id = np.argmax(paraphrased_text)
    label.append(id_to_label[max_id])


100%|██████████| 3426/3426 [1:46:33<00:00,  1.87s/it]


In [ ]:
df = pd.DataFrame({'id': test_data['id'].tolist(), 'label': label})
df.head()

,id,label
0,30670,Negative
1,4125,Negative
2,27077,Negative
3,17552,Negative
4,4137,Negative


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/BLP2023/Saved_dir/P-tuning/XLM_roberta-large/task.tsv', sep='\t', index=False)